In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from utilities.data_loaders import construct_embedding_matrix
from utilities.data_preprocessors import read_preprocess, series_to_1D_array

In [ ]:
# 1 for religious and 0 for non religious
df = pd.read_csv('./data/hate-speech-data-cleaned.csv', index_col=0)
df = read_preprocess(df)

In [ ]:
all_words = pd.Series(series_to_1D_array(df['comment']))
all_unique_words_counts = all_words.value_counts()
all_unique_words = all_words.unique()

In [ ]:
len(all_words)

In [ ]:
len(all_unique_words)

In [ ]:
all_unique_words_counts

In [ ]:
# before joining again get array in df with longest length first
max_len_1 = len(max(df['comment'], key=len))

In [ ]:
df['comment'] = df['comment'].apply(lambda comment: " ".join(comment))
df

In [ ]:
df.loc[0, 'comment']

**A note on the subsequent code below**

fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 0 is reserved for padding. So lower integer means more frequent word (often the first few are stop words because they appear a lot).

texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.

In [ ]:
# train_sents, test_sents, train_labels, test_labels = train_test_split(df['comment'], df['label'], test_size=0.3, random_state=0)
sents = df['comment']
max_len_2 = 50

num_words_1 = df.shape[0]
num_words_2 = len(all_words)
num_words_3 = len(all_unique_words)

tokenizer = Tokenizer(num_words=num_words_3, split=' ')
tokenizer.fit_on_texts(sents)
# the bug is here that's why there are wrong indeces

seqs = tokenizer.texts_to_sequences(sents)

# post means place padding of 0's on the tail or ending of the sequence
# and truncating removes the values of a sequence that is greater than the max length given
seqs_padded = pad_sequences(seqs, maxlen=max_len_1, padding='post', truncating='post')

In [ ]:
seqs

Here we see that indeed 50 is not enough as our max length but for the subsequent code we will still use 50 and later 503 for our experimentation. For now 503 will be an extremely large value eespecially when applied to all sequences

In [ ]:
print(max_len_1, max_len_2)

In [ ]:
word_to_index = tokenizer.word_index
index_to_word = tokenizer.index_word
print(len(word_to_index))

In [ ]:
word_to_index

In [ ]:
index_to_word

In [ ]:
seqs[0]

In [ ]:
# this is supposed to be 1301
print(word_index_dict['complain'])

# this is supposed to be 3583
print(word_index_dict['cleaning'])

In [ ]:
seqs_padded

In [ ]:
train_seqs, test_seqs, train_labels, test_labels = train_test_split(seqs_padded, df['label'], test_size=0.3, random_state=0)

train_seqs

In [ ]:
train_seqs.shape

In [ ]:
test_seqs

In [ ]:
len(test_seqs)

# Loading the Big Guns 
or the 1.9 million word vocabulary and its 300 dimensional embeddings

In [ ]:
# emb_matrix = construct_embedding_matrix('./embeddings/glove.42B.300d.txt', word_to_index, 300)


In [ ]:
emb_matrix[:5].shape

In [ ]:
num_non_exist = np.where(~emb_matrix.any(axis=1))[0]

In [ ]:
num_non_exist.shape